In [2]:
!pip install ortools

     |████████████████████████████████| 14.4MB 238kB/s 
     |████████████████████████████████| 1.0MB 31.4MB/s 
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


In [3]:
from ortools.linear_solver import pywraplp

In [33]:
from ortools.linear_solver import pywraplp
import numpy


def main():
    # Create the linear solver with the GLOP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Create the variables
    # x = numpy.zeros((N, U, K, M))
    x = {}
    for i in range(N):
        for u in range(U):
            for k in range(K):
                for j in range(M):
                    # x[i][u][k][j] = solver.IntVar(0,1,"x[" + str(i) + "," + str(u) +  "," + str(k) + "," + str(j) + "]")
                    x[i*U*K*M+u*K*M+k*M+j] = solver.IntVar(0, 1, 'x[{0},{1},{2},{3}]'.format(i,u,k,j))
                    # print('x[{0}, {1}, {2}, {3}]'.format(i,u,k,j))
                    # x[i][u][k][j] = solver.IntVar(0.0, 1, 'x')
    
    print('Number of variables =', solver.NumVariables())

    # Create a linear constraint, 0 <= x + y <= 2.
    # ct = solver.Constraint(0, 2, 'ct')
    # ct.SetCoefficient(x, 1)
    # ct.SetCoefficient(y, 1)
    # 1. Lớp học không thể xếp vào phòng có sức chứa nhỏ hơn số lượng sinh viên
    for i in range(N):
        for u in range(U):
            for k in range(K):
                for j in range(M):
                    ct = solver.Constraint(0, 0, 'ct1')
                    if s[i] > c[j]:
                        ct.SetCoefficient(x[i*U*K*M+u*K*M+k*M+j],1)

    # 2. Trong một thời điểm, một lớp chỉ có thể học tại 1 phòng
    for i in range(N):
        for u in range(U):
            for k in range(K):
                ct = solver.Constraint(0, 1, 'ct2')
                for j in range(M):
                    ct.SetCoefficient(x[i*U*K*M+u*K*M+k*M+j], 1)

    # 3. Trong cùng thời điểm, không thể có 2 lớp học cùng 1 phòng
    for i1 in range(N-1):
        for i2 in range(i1+1, N):
            ct = solver.Constraint(0, 1, 'ct3')
            for u in range(U):
                for k in range(K):
                    for j in range(M):
                        ct.SetCoefficient(x[i1*U*K*M+u*K*M+k*M+j], 1)
                        ct.SetCoefficient(x[i2*U*K*M+u*K*M+k*M+j], 1)
                        
    # 4. Trong cùng thời điểm, không thể có 2 lớp cùng giáo viên
    for i1 in range(N-1):
        for i2 in range(i1+1, N):
            ct = solver.Constraint(0, 1, 'ct4')
            if g[i1] == g[i2]:
                for u in range(U):
                    for k in range(K):
                        for j in range(M):
                            ct.SetCoefficient(x[i1*U*K*M+u*K*M+k*M+j], 1)
                            ct.SetCoefficient(x[i2*U*K*M+u*K*M+k*M+j], 1)

    # 5. Tổng tất cả các tiết học phải được bảo toàn
    T = 0
    for i in range(N):
        T += int(t[i])
    ct = solver.Constraint(0, T, 'ct5')
    for i in range(N):
        for u in range(U):
            for k in range(K):
                for j in range(M):
                    ct.SetCoefficient(x[i*U*K*M+u*K*M+k*M+j], 1)

    print('Number of constraints =', solver.NumConstraints())

    # Create the objective function, 3 * x + y.
    # objective = solver.Objective()
    # objective.SetCoefficient(x, 3)
    # objective.SetCoefficient(y, 1)
    # objective.SetMaximization()

    solver.Solve()

    print('Solution:')
    # print('Objective value =', objective.Value())
    # print('x =', x.solution_value())
    # print('y =', y.solution_value())
    for i in range(N):
        print('lop', i+1, ':')
        for u in range(U):
            for k in range(K):
                for j in range(M):
                    if(x[i*U*K*M+u*K*M+k*M+j].solution_value() == 1):
                        print(u+2,k+1,j+1)


def read_data(name='data.txt'):

    data = open(name, 'r')

    line_raw = data.readline().split('\n')
    line = line_raw[0].split(' ')
    global N, M, U, K, t, g, s, c
    N = int(line[0])
    M = int(line[1])
    U = 5
    K = 12
    t, g, s, c = [], [], [], []

    for i in range(N):
        line_raw = data.readline().split('\n')
        line = line_raw[0].split(' ')
        t.append(line[0])
        g.append(line[1])
        s.append(line[2])

    line_raw = data.readline().split('\n')
    line = line_raw[0].split(' ')
    for j in range(M):
        c.append(line[j])
    
    # print(N, M)
    # [print(t[i], g[i], s[i]) for i in range(N)]
    # [print(c[j], end=' ') for j in range(M)]
    data.close()



In [34]:
read_data(name='/content/drive/MyDrive/HUST/20202/Tối ưu lập kế hoạch/data_samples_mini_projects/miniproject-15/data.txt')
main()

Number of variables = 1800
Number of constraints = 2911
Solution:
lop 1 :
lop 2 :
lop 3 :
lop 4 :
lop 5 :
lop 6 :
lop 7 :
lop 8 :
lop 9 :
lop 10 :
lop 11 :
lop 12 :
lop 13 :
lop 14 :
lop 15 :
